<a href="https://colab.research.google.com/github/Henriquerezer/Machine_Learning/blob/main/Times_Series%5CLoop%5CTime_Series_for_Loop_Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importando Bibliotecas básicas

In [9]:
import pandas as pd #-> Leitura das tabelas
import numpy as np #-> Para operações
from prophet import Prophet #-> Série Temporal


Importação dos dados utilizando Pandas, podemos ver que a nossa demadna é diária, mas vamos precisar da nossa demanda **MENSAL**

In [10]:
data = pd.read_csv('/content/Data_for_time_series.csv', sep=';')
data

,DESC_APRESENTACAO,DIAS,UND,UF
0,A,2023-02-24,430.0,SE
1,A,2022-12-19,240.0,PI
2,A,2023-02-17,630.0,SC
3,A,2022-06-29,130.0,CE
4,A,2022-09-30,1030.0,BA
...,...,...,...,...
20143,C,2022-04-05,50.0,MG
20144,C,2022-11-28,50.0,AP
20145,C,2022-10-19,50.0,TO
20146,C,2023-02-02,100.0,GO


Nesta aplicação iremos utilizar Prophet para realizar a previsão de demanda de mais de um produto por estado, em um mesmo script, para que não seja necessário usar scripts diferentes. 

Neste caso, usamos um ``` for loop``` para iterar os valores de duas listas, ```itemlist``` e ```statelist```. O modelo é treinado com cada item destas lista. exemplo do processo do loop

1º  -> Item **A** e Estado **RS** 

2º  -> Modelo é treinado 

3º  -> Modelo gera uma previsão da demanda para 60 dias

4º  -> Itera próximo item e estado das duas listas

Deste modo os Items ``` A, B e C``` ser'ao iterados para cada estado, e teremos a previsão de demanda desses produtos para cada estado.


In [ ]:

gData = data.groupby(['DESC_APRESENTACAO', 'UF', 'DIAS'])['UND'].sum() # Agrupando por APRESENTAÇÃO/ UF / DIAS -> SOMANDO UNIDADES
gData = gData.to_frame().reset_index()
itemlist = gData.DESC_APRESENTACAO.unique()   # LISTA COM APRESENTAÇÕES
statelist = gData.UF.unique()                 # LISTA COM ESTADOS

fcst_all = pd.DataFrame()  # store all forecasts here
for x in itemlist:
     for y in statelist:
         temp = gData[(gData.DESC_APRESENTACAO == x) & (gData.UF == y)]
         temp = temp.drop(columns=['DESC_APRESENTACAO', 'UF'])
         temp['DIAS'] = pd.to_datetime(temp['DIAS'])
         temp = temp.set_index('DIAS')

         d_df = temp.resample('D').sum()
         d_df = d_df.reset_index().dropna()
         d_df.columns = ['ds', 'y']
         try:
             m = Prophet(seasonality_mode='additive',interval_width=0.95).fit(d_df)
             future = m.make_future_dataframe(periods=60, freq='D') # FORACAST PARA OS PRÓXIMOS 365 DIAS, FREQUENCIA DIARIA 
         except ValueError:
             pass       

         fcst = m.predict(future)
         fcst['Item'] = x  
         fcst['Estado'] = y
         fcst['Fact'] = d_df['y'].reset_index(drop = True)
         fcst_all = pd.concat((fcst_all, fcst))
         print(x, y)
#fcst_all # DATAFRAME COMPLETO


Pelo fato de precisarmos que nossa previsão da demanda seja mensal, iremos extrair o mês.

E por fim agruparmos nossos dados por Mês, Ano, Item e Estado, utilizando ```GROUPBY``` e iremos somar **yhat** ( nossa previsão) e **Fact** ( nossos dados reais)

In [12]:
fcst_all['ds'] = fcst_all['ds'].astype(str) # TRNAFORMADNDO COLUNA ds, PARA STRING
fcst_all['ds'] = pd.to_datetime(fcst_all['ds']) # TRNAFORMADNDO COLUNA ds, PARA DATETIME
fcst_all['mapped_month'] = fcst_all['ds'].dt.strftime('%B') # EXTRAINDO MÊS DA COLUNA ds, E CRIANDO UMA NOVA COLUNA
fcst_all['mapped_year'] = fcst_all['ds'].dt.strftime('%Y')  # EXTRAINDO ANO DA coluna ds, E CRIANDO UMA NOVA COLUNA
#fcst_all
data = fcst_all.groupby(by=['mapped_month', 'mapped_year', 'Item', 'Estado'])[['yhat', 'Fact']].sum().reset_index() # AGRUPANDO POR MÊS/ ANO/ APRESENTAÇÃO/ ESTADO e somando PROJEÇÕES / VALOR_REAL

Vizualizando Resultado

In [13]:
data.head()

,mapped_month,mapped_year,Item,Estado,yhat,Fact
0,April,2022,A,AC,103.065671,150.0
1,April,2022,A,AL,2458.398536,2420.0
2,April,2022,A,AM,11478.932012,11690.0
3,April,2022,A,AP,1916.572955,2330.0
4,April,2022,A,BA,3001.015541,1390.0


Conferindo data types

In [14]:
data.dtypes

mapped_month     object
mapped_year      object
Item             object
Estado           object
yhat            float64
Fact            float64
dtype: object

-------

**Métrica geral**

Agora iremos vizualizar a métrica geral do nosso modelo, sem levar em consideração a previsão para cada item iterado ou estado iterado


In [15]:
metricas = data[data['mapped_year'] == '2022'] # MÉTRICAS PARA O ANO DE 2022 (ANO EM QUE TEMOS VALORES DE Y_REAL)

In [16]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

r2 = r2_score(metricas['Fact'], metricas['yhat'])
rmse = mean_squared_error(metricas['Fact'], metricas['yhat'], squared=False)
MAPE = mean_absolute_percentage_error(metricas['Fact'], metricas['yhat'])
print('r2 score for a model which predicts mean value always is', r2)
print('RMSE score is: ', rmse)
print('MAPE score is: ',MAPE)

r2 score for a model which predicts mean value always is 0.9548846912615839
RMSE score is:  2321.1123375753164
MAPE score is:  2.862981291568703e+17


O R² (R-squared) é uma métrica que indica o quão bem o modelo se ajusta aos dados. Ele varia de 0 a 1, sendo que um valor próximo a 1 indica que o modelo se ajustou muito bem aos dados. No caso apresentado, o valor de R² é 0.9548846912615839, o que significa que o modelo se ajustou muito bem aos dados e conseguiu explicar 95,5% da variação observada nos dados.

O RMSE (Root Mean Squared Error) é uma medida de erro que indica a média da diferença entre os valores previstos pelo modelo e os valores reais observados nos dados. No caso apresentado, o valor de RMSE é 2321.1123375753164, o que significa que, em média, o modelo erra em cerca de 2321 unidades em relação aos valores reais.

-------
**Métricas de Todas as Produto**

Agora podemos ver as métricas para cada Produto, sem levar em consideração o estado

In [17]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
metricas = data[data['mapped_year'] == '2022'] # MÉTRICAS PARA O ANO DE 2022 (ANO EM QUE TEMOS VALORES DE Y_REAL)
# VISUALIZANDO AS MÉTRICAS DE TODAS AS APRESENTAÇÕES
items = metricas.Item.unique()
results = []
for item in items:
    filtered_metricas = metricas[metricas['Item'] == item]
    r2 = r2_score(filtered_metricas['Fact'], filtered_metricas['yhat'])
    rmse = mean_squared_error(filtered_metricas['Fact'], filtered_metricas['yhat'], squared=False)
    MAPE = mean_absolute_percentage_error(filtered_metricas['Fact'], filtered_metricas['yhat'])
    results.append([item, r2, rmse, MAPE])
    
result_df = pd.DataFrame(results, columns=['Item', 'r2', 'RMSE', 'MAPE'])
result_df

,Item,r2,RMSE,MAPE
0,A,0.983151,2053.082064,6.504381e+14
1,B,0.887137,2119.918849,2.339419e+15
2,C,0.771113,2728.274844,8.527259e+17


**Métricas de todas as Apresentações e todos os Estados**

Agora podemos analisar as métricas de cada produto para cada estado individualmente, e analisar e qual produto e estado o modelo está performando melhor e pior

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

metricas = data[data['mapped_year'] == '2022'] # MÉTRICAS PARA O ANO DE 2022 (ANO EM QUE TEMOS VALORES DE Y_REAL)

# VISUALIZANDO AS MÉTRICAS DE TODAS AS APRESENTAÇÕES
items = metricas.Item.unique()
estados = metricas.Estado.unique()

results = []
for item in items:
    for estado in estados:
        filtered_metricas = metricas[(metricas['Item'] == item) & (metricas['Estado'] == estado)]
        r2 = r2_score(filtered_metricas['Fact'], filtered_metricas['yhat'])
        rmse = mean_squared_error(filtered_metricas['Fact'], filtered_metricas['yhat'], squared=False)
        MAPE = mean_absolute_percentage_error(filtered_metricas['Fact'], filtered_metricas['yhat'])
        results.append([item, estado, r2, rmse, MAPE])

result_df2 = pd.DataFrame(results, columns=['Item', 'Estado', 'r2', 'RMSE', 'MAPE'])
result_df2.sort_values(by = 'r2', ascending = False)


,Item,Estado,r2,RMSE,MAPE
4,A,BA,0.875151,2485.166731,5.269836e-01
31,B,BA,0.765621,891.840598,6.418193e-01
10,A,MG,0.727145,935.482591,7.640521e-02
2,A,AM,0.703972,801.704806,6.058526e-02
58,C,BA,0.647554,1991.669569,8.182886e+00
...,...,...,...,...,...
5,A,CE,-0.605180,3410.418027,1.617113e-01
16,A,PR,-0.655457,423.687533,7.167421e-02
18,A,RN,-0.748907,997.336739,9.576943e-02
23,A,SE,-0.769231,1376.364666,7.730042e-02


----
Para facilitar a vizualização podemos criar uma coluna extra, informando se a previsão do próximo mês indica uma queda ou crescimento na demanda

Criando uma coluna identiicando se a demanda esta em crescimento ou em queda

In [18]:
data['mapped_year'] = data['mapped_year'].astype(int)
data = data[data['mapped_year'] == 2023]
meses = {'January' : 1, 'February' : 2, 'March' : 3, 'April' : 4, 'May' : 5, 'June' : 6, 'July' : 7, 'August' : 8, 'September' : 9, 'October' : 10, 'November' : 11, 'December' : 12}
data['mapped_month'] = data['mapped_month'].map(meses)
data = data.sort_values(by='mapped_month')
data = data.sort_values(by=['Item', 'Estado'], ignore_index = True)

<ipython-input-18-3fd6115ce772>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['mapped_month'] = data['mapped_month'].map(meses)


In [19]:
df = data
# Ordenando o dataframe pela coluna "Item" e "Estado"
df = df.sort_values(by=["Item", "Estado", "mapped_year", "mapped_month"])

# Criando uma nova coluna com a informação de Crescimento/Queda
df["Crescimento/Queda"] = ""

for index, row in df.iterrows():
    # Verificando se é a primeira linha para esse Item e Estado
    if index == 0 or (row[2:4].values != df.iloc[index-1, 2:4].values).any():
        df.at[index, "Crescimento/Queda"] = "Sem comparação"
    else:
        # Verificando se houve crescimento ou queda em relação à linha anterior
        if row["yhat"] > df.iloc[index-1]["yhat"]:
            df.at[index, "Crescimento/Queda"] = f"{(row['yhat']/df.iloc[index-1]['yhat']*100-100):.2f}% Em relação a demanda real do mês anterior"
        else:
            df.at[index, "Crescimento/Queda"] = f"{(row['yhat']/df.iloc[index-1]['yhat']*100-100):.2f}% Em relação a demanda real do mês anterior"



In [20]:
df[:12]

,mapped_month,mapped_year,Item,Estado,yhat,Fact,Crescimento/Queda
0,1,2023,A,AC,87.025967,110.0,Sem comparação
1,2,2023,A,AC,80.709605,110.0,-7.26% Em relação a demanda real do mês anterior
2,3,2023,A,AC,92.035239,0.0,14.03% Em relação a demanda real do mês anterior
3,4,2023,A,AC,81.998669,0.0,-10.91% Em relação a demanda real do mês anterior
4,1,2023,A,AL,2778.219692,3050.0,Sem comparação
5,2,2023,A,AL,2523.257902,2060.0,-9.18% Em relação a demanda real do mês anterior
6,3,2023,A,AL,2862.200908,0.0,13.43% Em relação a demanda real do mês anterior
7,4,2023,A,AL,2598.605593,0.0,-9.21% Em relação a demanda real do mês anterior
8,1,2023,A,AM,11538.339259,11830.0,Sem comparação
9,2,2023,A,AM,10626.173253,11360.0,-7.91% Em relação a demanda real do mês anterior


Aqui temos somente os 60 dias previstos pelo modelo, podemos ver que os dados do primeiro mês nós já temos como comparação **Fact**, podemos usar para vermos como o modelo está se saindo.

In [21]:
last_two_months = df['mapped_month'].max() - pd.Series([1, 2])
filtered_df = df.query('mapped_month in @last_two_months')
filtered_df

,mapped_month,mapped_year,Item,Estado,yhat,Fact,Crescimento/Queda
1,2,2023,A,AC,80.709605,110.0,-7.26% Em relação a demanda real do mês anterior
2,3,2023,A,AC,92.035239,0.0,14.03% Em relação a demanda real do mês anterior
5,2,2023,A,AL,2523.257902,2060.0,-9.18% Em relação a demanda real do mês anterior
6,3,2023,A,AL,2862.200908,0.0,13.43% Em relação a demanda real do mês anterior
9,2,2023,A,AM,10626.173253,11360.0,-7.91% Em relação a demanda real do mês anterior
...,...,...,...,...,...,...,...
313,3,2023,C,SE,6146.713144,0.0,15.24% Em relação a demanda real do mês anterior
316,2,2023,C,SP,57.111808,0.0,0.74% Em relação a demanda real do mês anterior
317,3,2023,C,SP,58.498449,0.0,2.43% Em relação a demanda real do mês anterior
319,2,2023,C,TO,774.229760,790.0,-10.95% Em relação a demanda real do mês anterior


Vizualizando um item em um estado

In [29]:
filtered_df[(filtered_df['Item'] == 'C') & (filtered_df['Estado'] == 'TO')]

,mapped_month,mapped_year,Item,Estado,yhat,Fact,Crescimento/Queda
319,2,2023,C,TO,774.229760,790.0,-10.95% Em relação a demanda real do mês anterior
320,3,2023,C,TO,870.807005,0.0,12.47% Em relação a demanda real do mês anterior
